# 一、赛题背景

随着全球经济的快速发展和城市化进程的加速，电力系统面临着越来越大的挑战。电力需求的准确预测对于电网的稳定运行、能源的有效管理以及可再生能源的整合至关重要。

![img](https://openres.xfyun.cn/xfyundoc/2024-06-29/0c19d023-bab0-4ee4-8bc3-20d670e67b92/1719632302187/107.png)

然而，电力需求受到多种因素的影响，为了提高电力需求预测的准确性和可靠性，推动智能电网和可持续能源系统的发展，本场以“电力需求预测”为赛题的数据算法挑战赛。选手需要根据历史数据构建有效的模型，能够准确的预测未来电力需求。

# 二、赛题任务

给定多个房屋对应电力消耗历史N天的相关序列数据等信息，预测房屋对应电力的消耗。

# 三、评审规则

## 1.数据说明

赛题数据由训练集和测试集组成，为了保证比赛的公平性，将每日日期进行脱敏，用1-N进行标识，即1为数据集最近一天，其中1-10为测试集数据。数据集由字段id（房屋id）、 dt（日标识）、type（房屋类型）、target（实际电力消耗）组成。

| 特征字段 | 字段描述               |
| -------- | ---------------------- |
| id       | 房屋id                 |
| dt       | 日标识                 |
| type     | 房屋类型               |
| target   | 实际电力消耗，预测目标 |

## 2.评审规则

预测结果以 mean square error 作为评判标准。

# 四、作品提交

1.文件格式：预测结果文件按照.csv格式提交

2.文件大小：无要求

3.提交次数：每支队伍每天最多3次

4.预测结果文件详细说明：

1）以csv格式提交，编码为UTF-8，第一行为表头；

2）提交前请确保预测结果的格式与sample_submit.csv中的格式一致。具体格式如下：

1 id,dt,target

2 00037f39cf,1,0

3 00037f39cf,2,0

4 00037f39cf,3,0

5 ...

In [63]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

In [64]:
train.head(10)

,id,dt,type,target
0,00037f39cf,11,2,44.050
1,00037f39cf,12,2,50.672
2,00037f39cf,13,2,39.042
3,00037f39cf,14,2,35.900
4,00037f39cf,15,2,53.888
5,00037f39cf,16,2,35.534
6,00037f39cf,17,2,41.280
7,00037f39cf,18,2,26.114
8,00037f39cf,19,2,25.612
9,00037f39cf,20,2,44.451


In [65]:
train_arr = []
train_id = []
for id, df in train.groupby('id'):
    df = df[['dt', 'target']]
    arr = df['target'].values[-200:]
    arr = np.pad(arr, (0, 200 - len(arr)), mode='constant', constant_values=0)
    train_arr.append(arr)
    train_id.append(id)

In [68]:
train_arr = np.array(train_arr)

In [69]:
model = LinearRegression()
model.fit(train_arr[:, :-10], train_arr[:, -10:])

LinearRegression()

In [71]:
pred = model.predict(train_arr[:, 10:])

In [77]:
test['target'] = pred.reshape(-1)

In [79]:
test[['id', 'dt', 'target']].to_csv('sub.csv', index=None)